In [2]:
# @title Imports (run this cell)
from __future__ import print_function

#importing the Libraries
import io
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from scipy import sparse
from scipy.stats import uniform


In [3]:
#load the dataset
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv


In [4]:
#Reading and Displaying the Movie Dataset
movie_dataset=pd.read_csv(io.StringIO(uploaded['movies.csv'].decode('utf-8')))
movie_dataset.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
#Reading and Displaying the Ratings Dataset
rate_dataset=pd.read_csv(io.StringIO(uploaded['ratings.csv'].decode('utf-8')))
rate_dataset.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [6]:
n_users = len(rate_dataset['userId'].unique())
n_items = len(rate_dataset['movieId'].unique())
R_shape = (n_users, n_items)
print (str(n_users) + ' users')
print (str(n_items) + ' items')

#668 Users Rated 10325 Movies

668 users
10325 items


In [0]:
#Genre List
genre_list = ["Action", "Adventure", "Animation", "Children", 
                "Comedy", "Crime","Documentary", "Drama", "Fantasy",
                "Film-Noir", "Horror", "Musical", "Mystery","Romance",
                "Sci-Fi", "Thriller", "War", "Western"]

In [8]:
#Creating the Search Matrix

movie_dataset['Action']=0
movie_dataset['Adventure']=0
movie_dataset['Animation']=0
movie_dataset['Children']=0
movie_dataset['Comedy']=0
movie_dataset['Crime']=0
movie_dataset['Documentary']=0
movie_dataset['Drama']=0
movie_dataset['Fantasy']=0
movie_dataset['Film-Noir']=0
movie_dataset['Horror']=0
movie_dataset['Musical']=0
movie_dataset['Mystery']=0
movie_dataset['Romance']=0
movie_dataset['Sci-Fi']=0
movie_dataset['Thriller']=0
movie_dataset['War']=0
movie_dataset['Western']=0

for i in movie_dataset.index:
  g_list=movie_dataset['genres'][i].split("|")
  for a in g_list:
    if a in genre_list:
      movie_dataset[a][i]=1

movie_dataset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
rate_dataset=rate_dataset.drop('timestamp',1)
rate_dataset.head()

,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0


In [10]:
#Sparse Matrix for the Ratings
col = ["userId","movieId"]
rating_one_hot = pd.get_dummies(rate_dataset,columns = col)
rating_one_hot.head()

,rating,userId_1,userId_2,userId_3,userId_4,userId_5,userId_6,userId_7,userId_8,userId_9,userId_10,userId_11,userId_12,userId_13,userId_14,userId_15,userId_16,userId_17,userId_18,userId_19,userId_20,userId_21,userId_22,userId_23,userId_24,userId_25,userId_26,userId_27,userId_28,userId_29,userId_30,userId_31,userId_32,userId_33,userId_34,userId_35,userId_36,userId_37,userId_38,userId_39,...,movieId_139920,movieId_140098,movieId_140110,movieId_140247,movieId_140265,movieId_140711,movieId_140715,movieId_140725,movieId_140737,movieId_140777,movieId_140816,movieId_140820,movieId_141305,movieId_141432,movieId_141472,movieId_141688,movieId_142074,movieId_142448,movieId_142488,movieId_142507,movieId_142536,movieId_142550,movieId_142562,movieId_142566,movieId_142973,movieId_143031,movieId_143257,movieId_143385,movieId_143709,movieId_143882,movieId_144482,movieId_144656,movieId_144976,movieId_146344,movieId_146656,movieId_146684,movieId_146878,movieId_148238,movieId_148626,movieId_149532
0,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
#Sparse_Matrix Alternative
row_ind = []
col_ind = []
data=[]
for ind in rate_dataset.index:
  x= rate_dataset['userId'][ind]
  y= rate_dataset['movieId'][ind]
  val=rate_dataset['rating'][ind]
  row_ind.append(x)
  col_ind.append(y)
  data.append(val)
  
rating_sparse = sparse.coo_matrix((data, (row_ind, col_ind)))

rating_matrix = pd.DataFrame.sparse.from_spmatrix(rating_sparse)
rating_matrix.head(40)
print(rating_matrix)

     0       1       2       3       ...  149529  149530  149531  149532
0       0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
1       0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
2       0.0     5.0     0.0     2.0  ...     0.0     0.0     0.0     0.0
3       0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4       0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
..      ...     ...     ...     ...  ...     ...     ...     ...     ...
664     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
665     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
666     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
667     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
668     0.0     3.0     3.0     2.0  ...     0.0     0.0     0.0     0.0

[669 rows x 149533 columns]


# Model-based Collaborative Filtering (Matrix Factorization approach)

### Non-Negative Matrix Factorisation Model

In [0]:
parametersNMF_opt = {
                    'n_components' : 20,     # number of latent factors
                    'init' : 'random', 
                    'random_state' : 0,
                     'alpha' : 0.01, 
                    'max_iter' : 25
                }

In [0]:
#Fitting the Model (Non-Negative Matrix Factorization)
from sklearn.decomposition import NMF
model = NMF(**parametersNMF_opt)
transform_matrix = model.fit_transform(rating_matrix)
M = model.components_.T

# Making the predictions
rating_pred = M.dot(transform_matrix.T).T
                    
# Clipping values                                                    
rating_pred[rating_pred > 5] = 5.           # clips ratings above 5             
rating_pred[rating_pred < 1] = 1.           # clips ratings below 1

In [14]:
merge=pd.merge(movie_dataset,rate_dataset)
merge.head()

,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,5.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,8,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,11,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,14,4.0


# Making Movie Recommendations for an Active User

In [15]:
def make_recommendation_activeuser(df,rating_matrix, prediction, user_idx, k=5):

    rated_items_df_user = pd.DataFrame(rating_matrix).iloc[user_idx, :]               # get the list of actual ratings of user_idx (seen movies)
    user_prediction_df_user = pd.DataFrame(prediction).iloc[user_idx,:]               # get the list of predicted ratings of user_idx (unseen movies)
    reco_df = pd.concat([rated_items_df_user, user_prediction_df_user], axis=1)       # merge both lists with the movie's title
    
   
    reco_df.columns = ['rating','prediction']

    
    # Return the 5 seen movies with the best actual Ratings
    print ('Preferred movies for user #', user_idx)                                   
    l=[]
    l= reco_df.sort_values(by='rating', ascending=False)[:k].index
    
    m=[]
    for a in l:
      m.append(df.loc[df['movieId'] == a, 'title'].iloc[0])
    concat1=reco_df.sort_values(by='rating', ascending=False)[:k]
    concat1['TITLE']=m
    print (concat1)
    
    #Return the 5 unseen movie with the best predicted Ratings
    print ('Recommended movies for user #', user_idx)   # returns the 5 unseen movies with the best predicted ratings
    reco_df = reco_df[ reco_df['rating'] == 0 ]
    x=[]
    x=reco_df.sort_values(by='prediction', ascending=False)[:k].index     
    
    y=[]
    for b in x:
      y.append(df.loc[df['movieId'] == b, 'title'].iloc[0])
    concat2=reco_df.sort_values(by='prediction', ascending=False)[:k]
    concat2['TITLE']=y
    print(concat2)
    print

make_recommendation_activeuser(movie_dataset,rating_matrix, rating_pred, user_idx=50, k=5)
make_recommendation_activeuser(movie_dataset,rating_matrix, rating_pred, user_idx=2, k=5)

Preferred movies for user # 50
      rating  prediction                                             TITLE
36       5.0    1.000000                           Dead Man Walking (1995)
306      5.0    1.000000  Three Colors: Red (Trois couleurs: Rouge) (1994)
1394     5.0    1.220711                            Raising Arizona (1987)
1237     5.0    1.000000   Seventh Seal, The (Sjunde inseglet, Det) (1957)
493      5.0    1.000000                          Menace II Society (1993)
Recommended movies for user # 50
      rating  prediction                           TITLE
2858     NaN    1.732541          American Beauty (1999)
1221     NaN    1.404439  Godfather: Part II, The (1974)
912      NaN    1.387566               Casablanca (1942)
2997     NaN    1.362382     Being John Malkovich (1999)
111      NaN    1.342709              Taxi Driver (1976)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:2418: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  result = result.to_dense()


Preferred movies for user # 2
     rating  prediction                      TITLE
608     5.0    1.914664               Fargo (1996)
805     5.0    1.661544     Time to Kill, A (1996)
36      5.0    1.611408    Dead Man Walking (1995)
1       5.0    2.722292           Toy Story (1995)
62      5.0    2.004091  Mr. Holland's Opus (1995)
Recommended movies for user # 2
      rating  prediction                                              TITLE
733      NaN    2.613782                                   Rock, The (1996)
786      NaN    2.079849                                      Eraser (1996)
1210     NaN    2.059246  Star Wars: Episode VI - Return of the Jedi (1983)
832      NaN    2.013805                                      Ransom (1996)
141      NaN    1.965273                               Birdcage, The (1996)
